<a href="https://colab.research.google.com/github/po3rin/python_playground/blob/master/bert_ner/bert_ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


In [2]:
!pip install seqeval transformers==3.1.0

     |████████████████████████████████| 51kB 4.2MB/s 
     |████████████████████████████████| 890kB 10.4MB/s 
     |████████████████████████████████| 1.2MB 27.6MB/s 
     |████████████████████████████████| 3.0MB 34.4MB/s 
     |████████████████████████████████| 890kB 44.2MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16172 sha256=88087fc320f76bd6835d9d295ff2b20acd3e24cc540be0f23930b1c0512c574c
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=bc2d532a8c7bf66aaa9dc21684d1c5b12d01e4c52bc0eb581dfe5562bfc9d35e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built seqeval sacremoses


In [3]:
!mkdir data
!mkdir models
!wget https://raw.githubusercontent.com/Hironsan/IOB2Corpus/master/ja.wikipedia.conll -P data/

--2021-02-28 13:46:10--  https://raw.githubusercontent.com/Hironsan/IOB2Corpus/master/ja.wikipedia.conll
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1297592 (1.2M) [text/plain]
Saving to: ‘data/ja.wikipedia.conll’

ja.wikipedia.conll  100%[===================>]   1.24M  --.-KB/s    in 0.05s   

2021-02-28 13:46:10 (26.2 MB/s) - ‘data/ja.wikipedia.conll’ saved [1297592/1297592]



In [4]:
batch_size = 32
epochs = 100
num_words = 15000

In [5]:
import re

import numpy as np
import tensorflow as tf
from seqeval.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import BertJapaneseTokenizer

In [6]:
def evaluate(model, target_vocab, features, labels):
    label_ids = model.predict(features)[0]
    label_ids = np.argmax(label_ids, axis=-1)
    y_pred = [[] for _ in range(label_ids.shape[0])]
    y_true = [[] for _ in range(label_ids.shape[0])]
    for i in range(label_ids.shape[0]):
        for j in range(label_ids.shape[1]):
            if labels[i][j] == 0:
                continue
            y_pred[i].append(label_ids[i][j])
            y_true[i].append(labels[i][j])
    y_pred = target_vocab.decode(y_pred)
    y_true = target_vocab.decode(y_true)
    print(classification_report(y_true, y_pred, digits=4))

In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences


def convert_examples_to_features(x, y,
                                 vocab,
                                 max_seq_length,
                                 tokenizer):
    pad_token = 0
    features = {
        'input_ids': [],
        'attention_mask': [],
        'token_type_ids': [],
        'label_ids': []
    }
    for words, labels in zip(x, y):
        tokens = [tokenizer.cls_token]
        label_ids = [pad_token]
        for word, label in zip(words, labels):
            word_tokens = tokenizer.tokenize(word)
            tokens.extend(word_tokens)
            label_id = vocab.get_index(label)
            label_ids.extend([label_id] + [pad_token] * (len(word_tokens) - 1))
        tokens += [tokenizer.sep_token]

        input_ids = tokenizer.convert_tokens_to_ids(tokens)
        attention_mask = [1] * len(input_ids)
        token_type_ids = [pad_token] * max_seq_length

        features['input_ids'].append(input_ids)
        features['attention_mask'].append(attention_mask)
        features['token_type_ids'].append(token_type_ids)
        features['label_ids'].append(label_ids)

    for name in features:
        features[name] = pad_sequences(features[name], padding='post', maxlen=max_seq_length)

    x = [features['input_ids'], features['attention_mask'], features['token_type_ids']]
    y = features['label_ids']
    return x, y

In [9]:
import tensorflow as tf
from transformers import TFBertForTokenClassification, BertConfig


def build_model(pretrained_model_name_or_path, num_labels):
    config = BertConfig.from_pretrained(
        pretrained_model_name_or_path,
        num_labels=num_labels
    )
    model = TFBertForTokenClassification.from_pretrained(
        pretrained_model_name_or_path,
        config=config
    )
    model.layers[-1].activation = tf.keras.activations.softmax
    return model


def loss_func(num_labels):
    loss_fct = tf.keras.losses.SparseCategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)

    def loss(y_true, y_pred):
        input_mask = tf.not_equal(y_true, 0)
        logits = tf.reshape(y_pred, (-1, num_labels))
        active_loss = tf.reshape(input_mask, (-1,))
        active_logits = tf.boolean_mask(logits, active_loss)
        train_labels = tf.reshape(y_true, (-1,))
        active_labels = tf.boolean_mask(train_labels, active_loss)
        cross_entropy = loss_fct(active_labels, active_logits)
        return cross_entropy
    return loss

In [10]:
def load_dataset(filename, encoding='utf-8'):
    """Loads data and label from a file.
    Args:
        filename (str): path to the file.
        encoding (str): file encoding format.
        The file format is tab-separated values.
        A blank line is required at the end of a sentence.
        For example:
        ```
        EU	B-ORG
        rejects	O
        German	B-MISC
        call	O
        to	O
        boycott	O
        British	B-MISC
        lamb	O
        .	O
        Peter	B-PER
        Blackburn	I-PER
        ...
        ```
    Returns:
        tuple(numpy array, numpy array): data and labels.
    Example:
        >>> filename = 'conll2003/en/ner/train.txt'
        >>> data, labels = load_data_and_labels(filename)
    """
    sents, labels = [], []
    words, tags = [], []
    with open(filename, encoding=encoding) as f:
        for line in f:
            line = line.rstrip()
            if line:
                word, tag = line.split('\t')
                words.append(word)
                tags.append(tag)
            else:
                sents.append(words)
                labels.append(tags)
                words, tags = [], []
        if words:
            sents.append(words)
            labels.append(tags)

    return sents, labels

In [11]:
class Vocab:

    def __init__(self, num_words=None, lower=True, oov_token=None):
        self.tokenizer = tf.keras.preprocessing.text.Tokenizer(
            num_words=num_words,
            oov_token=oov_token,
            filters='',
            lower=lower,
            split='\t'
        )

    def fit(self, sequences):
        texts = self._texts(sequences)
        self.tokenizer.fit_on_texts(texts)
        return self

    def encode(self, sequences):
        texts = self._texts(sequences)
        return self.tokenizer.texts_to_sequences(texts)

    def decode(self, sequences):
        texts = self.tokenizer.sequences_to_texts(sequences)
        return [text.split(' ') for text in texts]

    def _texts(self, sequences):
        return ['\t'.join(words) for words in sequences]

    def get_index(self, word):
        return self.tokenizer.word_index.get(word)

    @property
    def size(self):
        """Return vocabulary size."""
        return len(self.tokenizer.word_index) + 1

    def save(self, file_path):
        with open(file_path, 'w') as f:
            config = self.tokenizer.to_json()
            f.write(config)

    @classmethod
    def load(cls, file_path):
        with open(file_path) as f:
            tokenizer = tf.keras.preprocessing.text.tokenizer_from_json(f.read())
            vocab = cls()
            vocab.tokenizer = tokenizer
        return vocab


def normalize_number(text, reduce=True):
    if reduce:
        normalized_text = re.sub(r'\d+', '0', text)
    else:
        normalized_text = re.sub(r'\d', '0', text)
    return normalized_text


def preprocess_dataset(sequences):
    sequences = [[normalize_number(w) for w in words] for words in sequences]
    return sequences


def create_dataset(sequences, vocab):
    sequences = vocab.encode(sequences)
    sequences = pad_sequences(sequences, padding='post')
    return sequences

In [ ]:
def main():
    # Set hyper-parameters.
    batch_size = 16
    epochs = 100
    model_path = 'models/'
    pretrained_model_name_or_path = 'cl-tohoku/bert-base-japanese-whole-word-masking'
    maxlen = 250

    # Data loading.
    x, y = load_dataset('./data/ja.wikipedia.conll')
    tokenizer = BertJapaneseTokenizer.from_pretrained(pretrained_model_name_or_path, do_word_tokenize=False)

    # Pre-processing.
    x = preprocess_dataset(x)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    target_vocab = Vocab(lower=False).fit(y_train)
    features_train, labels_train = convert_examples_to_features(
        x_train,
        y_train,
        target_vocab,
        max_seq_length=maxlen,
        tokenizer=tokenizer
    )
    features_test, labels_test = convert_examples_to_features(
        x_test,
        y_test,
        target_vocab,
        max_seq_length=maxlen,
        tokenizer=tokenizer
    )

    # Build model.
    model = build_model(pretrained_model_name_or_path, target_vocab.size)
    model.compile(optimizer='sgd', loss=loss_func(target_vocab.size))

    # Preparing callbacks.
    callbacks = [
        EarlyStopping(patience=3),
    ]

    # Train the model.
    model.fit(x=features_train,
              y=labels_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_split=0.1,
              callbacks=callbacks,
              shuffle=True)
    model.save_pretrained(model_path)

    # Evaluation.
    evaluate(model, target_vocab, features_test, labels_test)


if __name__ == '__main__':
    main()

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing TFBertForTokenClassification: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForTokenClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier', 'dropout_37']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/100
45/45 [==============================] - 2015s 45s/step - loss: 1.1680 - val_loss: 0.6656
Epoch 2/100
22/45 [=============>................] - ETA: 16:43 - loss: 0.6537

In [ ]:
! ls models

In [ ]:
model = load_model(model_path)
label_ids = model.predict(features)[0]
label_ids = np.argmax(label_ids, axis=-1)
y_pred = [[] for _ in range(label_ids.shape[0])]
print(y_pred)